In [22]:
import pandas as pd
from tqdm import tqdm
import re
import roman

In [23]:
checkliste = ['Kupfert',
              'Tit',
              'Titel',
              'Kupfertitel',
              'Titelbl',
              'Kupfertit',
              'Bildertit',
              'Frontisp',
              'Kupferfrontispiz',
              'Kupferfrontisp',
              'Porträtkupf',
              'Vort',
              'Portr.-Kupf.',
              'Frontispiz',
              'Front',
              'Port. Kupf.']

In [3]:
df = pd.read_csv('buchbestand.csv')
df

,idn,year,verlag_ort,herst_ort_idn,herst_ort_name,umfang,format
0,010003266,1886,Hannover,NaN,NaN,NaN,NaN
1,010006435,1965,Leipzig,NaN,NaN,NaN,NaN
2,010010335,1960,München,NaN,NaN,NaN,NaN
3,010010939,1913,Bern,NaN,NaN,NaN,30 cm
4,010013431,1934,Stockholm [u.a.],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
148129,1235244601,2021,NaN,NaN,NaN,NaN,NaN
148130,1235245594,2021,NaN,NaN,NaN,NaN,NaN
148131,123524606X,2021,NaN,NaN,NaN,NaN,NaN
148132,1235250016,1894,Berlin,040057283,Berlin,13 Seiten,NaN


In [32]:
def seiten_ermitteln(format):
    seiten = 0
    fehlseiten = []
    merkliste = []
    try:
        for element in format.split(','):
            #print(element)
            if re_find := re.search('\[?(\d+)\]?\s.*\s?(Bl|Sp|S|Faltbl|Taf|Kupf|Blätter|Seiten|Spalten).*', element.strip()):
                    if re_find.group(2) == 'Bl' or re_find.group(2) == 'Faltbl' or re_find.group(2) == 'Taf' or re_find.group(2) == 'Blätter' or re_find.group(2) == 'Kupf':
                        seiten += int(re_find.group(1)) * 2
                        #print('finde bl')
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl * 2
                            merkliste = []
                    elif re_find.group(2) == 'S' or re_find.group(2) == 's' or re_find.group(2) == 'Seiten':
                        #print('finde s')
                        seiten += int(re_find.group(1))
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl
                            merkliste = []
                    elif re_find.group(2) == 'Sp' or re_find.group(2) == 'Spalten':
                        seiten += int(re_find.group(1))/4
                        #print(f'{re_find.group(1)} Spalten = {str(int(re_find.group(1))/4)} Seiten')
                    else:
                        None
            elif re_find := re.search('(^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3}))(\s(?P<typ>S|Bl|Seiten|Blätter|Taf|Tafeln\.))?.*$', element.strip()):
                    if re_find.group('typ')  == 'S' or re_find.group('typ') == 'Seiten':
                        seiten += roman.fromRoman(re_find.group(1).strip())
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl
                            merkliste = []

                    elif re_find.group('typ')  == 'Bl' or re_find.group('typ') == 'Blätter' or re_find.group('typ')== 'Taf' or re_find.group('typ') == 'Tafeln':
                        seiten += roman.fromRoman(re_find.group(1).strip()) * 2
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl * 2
                            merkliste = []
                    else:
                        merkliste.append(roman.fromRoman(re_find.group(1).strip()))
            elif re_find := re.search('(?P<typ>S\.|Seiten)\s?(?P<von>\d+)\s?-\s?(?P<bis>\d+)', element.strip()):
                    seiten += int(re_find.group('bis')) - int(re_find.group('von'))
                    if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl
                            merkliste = []
            elif re_find := re.search('Bl.\s?(\d+)\s?-\s?(\d+)', element.strip()):
                    seiten += (int(re_find.group(2)) - int(re_find.group(1))) * 2
                    if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl * 2
                            merkliste = []
            elif re_find := re.search('Sp.\s?(\d+)\s?-\s?(\d+)', element.strip()):
                    seiten += (int(re_find.group(2)) - int(re_find.group(1))) / 2
                    if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl / 4
                            merkliste = []
            elif any(i in element.strip() for i in checkliste):
                    seiten += 2
            elif re_find := re.match('\[?(\d+)\]?', element.strip()):
                    merkliste.append(int(re_find.group(1)))
            else:
                fehlseiten.append(element)
    except Exception as e:
        seiten = 0
        #print(e)
    return int(seiten)#, fehlseiten

In [33]:
sample = df[pd.notna(df.umfang)].sample(100)
sample

,idn,year,verlag_ort,herst_ort_idn,herst_ort_name,umfang,format,seiten
81131,986394785,2007,München,NaN,NaN,"VIII, 375 S.",23 cm,0
55122,931214203,1992,New York,NaN,NaN,"IX, 146 S.",30 cm,0
57174,943813204,1993,Frankfurt am Main,NaN,NaN,107 S.,25 cm,0
23776,572179529,1868,Pest,NaN,NaN,256 S.,gr. 8,0
35067,579407845,1948,Berlin,NaN,NaN,8 S.,8,0
...,...,...,...,...,...,...,...,...
58319,946558280,1996,Innsbruck,947417001,Himberg,286 S.,31 cm,0
14492,450850692,1947,Wien,NaN,NaN,38 S.,8,0
128432,1066972036,1474,[Köln],040314839,Köln,174 Bl.,2°,0
69659,971881332,2004,[Paderborn],NaN,NaN,199 S.,"24 cm, 377 gr.",0


In [34]:
sample['seiten'] = sample.umfang.apply(seiten_ermitteln)
sample

,idn,year,verlag_ort,herst_ort_idn,herst_ort_name,umfang,format,seiten
81131,986394785,2007,München,NaN,NaN,"VIII, 375 S.",23 cm,383
55122,931214203,1992,New York,NaN,NaN,"IX, 146 S.",30 cm,155
57174,943813204,1993,Frankfurt am Main,NaN,NaN,107 S.,25 cm,107
23776,572179529,1868,Pest,NaN,NaN,256 S.,gr. 8,256
35067,579407845,1948,Berlin,NaN,NaN,8 S.,8,8
...,...,...,...,...,...,...,...,...
58319,946558280,1996,Innsbruck,947417001,Himberg,286 S.,31 cm,286
14492,450850692,1947,Wien,NaN,NaN,38 S.,8,38
128432,1066972036,1474,[Köln],040314839,Köln,174 Bl.,2°,348
69659,971881332,2004,[Paderborn],NaN,NaN,199 S.,"24 cm, 377 gr.",199


In [35]:
df['seiten'] = df.umfang.apply(seiten_ermitteln)
df

,idn,year,verlag_ort,herst_ort_idn,herst_ort_name,umfang,format,seiten
0,010003266,1886,Hannover,NaN,NaN,NaN,NaN,0
1,010006435,1965,Leipzig,NaN,NaN,NaN,NaN,0
2,010010335,1960,München,NaN,NaN,NaN,NaN,0
3,010010939,1913,Bern,NaN,NaN,NaN,30 cm,0
4,010013431,1934,Stockholm [u.a.],NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
148129,1235244601,2021,NaN,NaN,NaN,NaN,NaN,0
148130,1235245594,2021,NaN,NaN,NaN,NaN,NaN,0
148131,123524606X,2021,NaN,NaN,NaN,NaN,NaN,0
148132,1235250016,1894,Berlin,040057283,Berlin,13 Seiten,NaN,13
